In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

In [3]:
def df_with_obj(df, n):
    return df[df['obj_init'] == n]

def is_successful(df, thr=30):
    return df.shape[0] < thr

def success_rate(df):
    suc = df[['run']].groupby(['run']).apply(is_successful).sum()
    total = pd.unique(df['run']).shape[0]
    
    return suc / total

def avg_successful_pushes(df):
    cnt = df.groupby(['run']).size().reset_index(name='counts')
    cnt = cnt['counts'].sum()
    
    total = pd.unique(df['run']).shape[0]
    
    return cnt / total

def sum_mask_rewards(df):
    sm = df.groupby(['run'])    
    sm = sm['diff_seg_reward'].sum()
    sm = sm.sum()
    
    total = pd.unique(df['run']).shape[0]

    return sm / total